In [2]:
import SimpleITK as sitk
import numpy as np
import csv
from glob import glob
import pandas as pd

# path constants
LUNA_DATA_PATH = '../../../../data/luna16/'
LUNA_SUBSET_PATH = LUNA_DATA_PATH + 'subset0/'

file_list = glob(LUNA_SUBSET_PATH + "*.mhd") # get all the mhd image files

# Helper function to get rows in data frame associated with each file
def get_filename(case):
    global file_list
    for f in file_list: # for every file in the list if the seriesuid is in the file name, return the file 
        if case in f:
            return(f)

# The locations of the nodes
df_node = pd.read_csv(LUNA_DATA_PATH + "annotations.csv")
df_node["file"] = df_node["seriesuid"].apply(get_filename) # for every rowsave file name to the 'file' column of the row
df_node = df_node.dropna() # if the seriesuid is not found in this subset, drop all the rows that have na as values for 'file' column


mini_df = df_node[df_node["file"]==file_list[2]]
print(mini_df)
print('\n')
print(mini_df["diameter_mm"].values)
print('\n')
print(np.argsort(mini_df["diameter_mm"].values))
print('\n')
print(np.argsort(mini_df["diameter_mm"].values)[-1])

                                            seriesuid     coordX     coordY  \
25  1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524...  46.188539  48.402806   
26  1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524...  36.392044  76.771663   

        coordZ  diameter_mm                                               file  
25 -108.578632    13.596471  ../../../../data/luna16/subset0\1.3.6.1.4.1.14...  
26 -123.321911     4.343200  ../../../../data/luna16/subset0\1.3.6.1.4.1.14...  


[ 13.59647134   4.34319974]


[1 0]


0
